In [2]:
import pickle
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import os
from collections import defaultdict
from os import listdir
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import lda
import numpy as np
from operator import itemgetter
from collections import Counter

In [15]:
mypath='unsup/'
def getMovieDat():
    
    #documents = [f for f in listdir(mypath) if f.startswith('data_')]
    #documents = [os.listdir(mypath)]
    documents = [f for f in listdir(mypath) if f.endswith('.txt')]
    lmtzr = WordNetLemmatizer()
    docs=[]
    sent_corp=defaultdict(list)
    tokenizer = RegexpTokenizer(r'\w+')
    wrd_freq=Counter()
    for doc in tqdm(documents):
        with open(mypath+doc,'r', errors = 'ignore') as fp:
            lines = ' '.join(fp.readlines())
            sent = sent_tokenize(lines)
            tmp = ''
            for sen in sent:
                tokens=tokenizer.tokenize(sen)
                tagged = nltk.pos_tag(tokens)
                nouns = [word for word,pos in tagged \
                         if (pos == 'NN' or pos == 'NNP' or pos == 'NNS'\
                             or pos == 'NNPS' or pos=='JJ')]
                downcased = [x.lower() for x in nouns]
                downcased = [lmtzr.lemmatize(x) for x in downcased]
                downcased = [x for x in downcased if x not in stopwords.words('english')]
                if len(downcased)>1:
                    sent_corp[doc].append([set(downcased)]+[sen])
                    tmp += ' '.join(downcased)
                    for x in downcased: wrd_freq[x]+=1
            docs.append(tmp)
    tp_wrds=sorted(wrd_freq.items(), key=itemgetter(1), reverse=True)
    return (docs,tp_wrds,sent_corp)
docs,tp_wrds,sent_corp = getMovieDat()

100%|████████████████████████████████████████████████████████████████████████████| 50000/50000 [48:40<00:00, 17.12it/s]


In [20]:
pickle.dump( docs, open( "docs.pkl", "wb" ) )

In [10]:
docs = pickle.load( open( "docs.pkl", "rb" ) )

In [11]:
docs[0]

'great majority film saydozen major silents crowd good last command city light latter chaplin circa br br apprehensive humor difficult uh enjoy decadelead actor film br intriguing sequenceguy minute schtickbackground dozen men naked white black wwibutt part full backsidewa early variation beefcake courtesy howard hughes'

In [12]:
# run this block converts words to integers for LDA input
def getMovNAmazonDat(docs):
     
    # change the location of data here
    vectorizer = CountVectorizer(min_df=1,stop_words='english')
    reditposts=[d for d in docs]
 
    rv_vec=vectorizer.fit_transform(reditposts)
    f_nam=tuple(vectorizer.get_feature_names())     
    return(rv_vec,f_nam,docs)

train_d,f_nam,docs = getMovNAmazonDat(docs)

In [25]:
# train the traditional LDA model
model = lda.LDA(n_topics=500, n_iter=1000, random_state=1)
model.fit(train_d)
topic_word = model.topic_word_

INFO:lda:n_documents: 50000
INFO:lda:vocab_size: 399380
INFO:lda:n_words: 3467369
INFO:lda:n_topics: 500
INFO:lda:n_iter: 1000
C:\Anaconda\lib\site-packages\lda\utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -60726348
INFO:lda:<10> log likelihood: -45430808
INFO:lda:<20> log likelihood: -42909913
INFO:lda:<30> log likelihood: -41549608
INFO:lda:<40> log likelihood: -40845687
INFO:lda:<50> log likelihood: -40453785
INFO:lda:<60> log likelihood: -40196098
INFO:lda:<70> log likelihood: -40015332
INFO:lda:<80> log likelihood: -39884683
INFO:lda:<90> log likelihood: -39794660
INFO:lda:<100> log likelihood: -39705187
INFO:lda:<110> log likelihood: -39648534
INFO:lda:<120> log likelihood: -39597428
INFO:lda:<130> log likelihood: -39552747
INFO:lda:<140> log likelihood

In [27]:
pickle.dump( model, open( "model.pkl", "wb" ) )

In [3]:
model = pickle.load( open( "model.pkl", "rb" ) )

In [5]:
import pandas as pd
topic_doc = pd.DataFrame(model.doc_topic_)
maxprob = []
maxprob = topic_doc.apply(lambda x: x.argmax(), axis=1)

In [7]:
topic_word = model.topic_word_
doc_topic = model.doc_topic_

In [27]:
# qry=set(['screen','brightness','display','lcd','reflectiveness'])
qry=set(['logan','apology','phone','parody','artist'])
n_top_words = 20
top_word_list = []
for i, topic_dist in tqdm(enumerate(topic_word)):
    topic_words = np.array(f_nam)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    tmp=set(topic_words)
    #if qry.intersection(tmp):
     #   print('Topic {}: {}'.format(i, ' '.join(topic_words)))
    top_word_list.append('Topic {}: {}'.format(i, ' '.join(topic_words)))

500it [02:42,  3.08it/s]


In [28]:
result=[]
for i in maxprob:
    result.append(top_word_list[i])

In [30]:
result[0]

'Topic 46: chaplin comedy little hardy film laurel tramp charlie early short stan circus trouble charley sequence silent gag ollie oliver studio'

In [17]:
pickle.dump( result, open( "resultsml.pkl", "wb" ) )

In [25]:
data1 = "I admit, the great majority of films released before say 1933 are just not for me. Of the dozen or so 'major' silents I have viewed, one I loved (The Crowd), and two were very good (The Last Command and City Lights, that latter Chaplin circa 1931).<br /><br />So I was apprehensive about this one, and humor is often difficult to appreciate (uh, enjoy) decades later. I did like the lead actors, but thought little of the film.<br /><br />One intriguing sequence. Early on, the guys are supposed to get 'de-loused' and for about three minutes, fully dressed, do some schtick. In the background, perhaps three dozen men pass by, all naked, white and black (WWI ?), and for most, their butts, part or full backside, are shown. Was this an early variation of beefcake courtesy of Howard Hughes?"

In [26]:
data1

"I admit, the great majority of films released before say 1933 are just not for me. Of the dozen or so 'major' silents I have viewed, one I loved (The Crowd), and two were very good (The Last Command and City Lights, that latter Chaplin circa 1931).<br /><br />So I was apprehensive about this one, and humor is often difficult to appreciate (uh, enjoy) decades later. I did like the lead actors, but thought little of the film.<br /><br />One intriguing sequence. Early on, the guys are supposed to get 'de-loused' and for about three minutes, fully dressed, do some schtick. In the background, perhaps three dozen men pass by, all naked, white and black (WWI ?), and for most, their butts, part or full backside, are shown. Was this an early variation of beefcake courtesy of Howard Hughes?"

In [31]:
result[0]

'Topic 46: chaplin comedy little hardy film laurel tramp charlie early short stan circus trouble charley sequence silent gag ollie oliver studio'